# IMDb Movie Rating classifier

In [1]:
# Importing the libraries :

import csv

import numpy as np

import tensorflow as tf

from sklearn.preprocessing import LabelEncoder

from tensorflow.keras.preprocessing.text import Tokenizer

from tensorflow.keras.preprocessing.sequence import pad_sequences

# Loading the dataste 

In [2]:
file_path = r"C:\Users\PMLS\Desktop\Personal\Coursera\NLP\IMDB_movie_rating\IMDB Dataset.csv\IMDB Dataset.csv"

with open(file_path, "r") as csvfile:
    
    csv_reader = csv.reader(csvfile)

    print(f"First line (Header) Looks like this :\n\n       {csvfile.readline()}")
    
    print(f'Each Data point looks like this :\n\n           {csvfile.readline()}')
        

First line (Header) Looks like this :

       review,sentiment

Each Data point looks like this :

           "One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealing

In [3]:
training_file_path = r"C:\Users\PMLS\Desktop\Personal\Coursera\NLP\IMDB_movie_rating\IMDB Dataset.csv\train.csv"

test_file_path = r"C:\Users\PMLS\Desktop\Personal\Coursera\NLP\IMDB_movie_rating\IMDB Dataset.csv\test.csv"

In [4]:
# loading training data

training_sentences = []

training_labels = []


with open(training_file_path, 'r', encoding='utf-8') as csvfile:
    
    reader = csv.reader(csvfile)
    
    header = next(reader, None)
    
    for row in reader:
        
        review, sentiment = row
        
        training_sentences.append(review)
        
        training_labels.append(sentiment)
        

In [5]:
# Loading test data :

test_sentences = []

test_labels = []

with open(test_file_path, 'r', encoding='utf-8') as csvfile:
    
    reader = csv.reader(csvfile)
    
    header = next(reader, None)
    
    for row in reader:
        
        review, sentiment = row
        
        test_sentences.append(review)
        
        test_labels.append(sentiment)

In [6]:
# Assigning labels a numeric value :

label_encoder = LabelEncoder()

training_labels_encoded = label_encoder.fit_transform(training_labels)

test_labels_encoded = label_encoder.fit_transform(test_labels)

# Convert labels into numpy arrays :

training_labels_final = np.array(training_labels_encoded)

test_labels_final = np.array(test_labels_encoded)

In [7]:
print(training_labels_final, test_labels_final)

[1 1 1 ... 0 0 1] [0 1 1 ... 0 0 0]


# Generate Padded Sequence

In [8]:
# parameters:

vocabulary_size = 10000

max_length = 120

embedding_dim = 16      # Dimensions to represent dense vectors

trunc_type = 'post'     # Making the length of all sentences equal

oov_tok = '<OOV>'       # Out of vocabulary tokens

In [9]:
# Initialize tokenizer class :

tokenizer = Tokenizer(num_words= vocabulary_size, oov_token = oov_tok)

# GEnerate the word index :

tokenizer.fit_on_texts(training_sentences)

word_index = tokenizer.word_index

# Generate and Pad the training sequence :

sequences = tokenizer.texts_to_sequences(training_sentences)

padded = pad_sequences(sequences , maxlen=max_length , truncating=trunc_type)

# Generate the test sequence :

test_sequences = tokenizer.texts_to_sequences(test_sentences)

test_padded = pad_sequences(test_sequences , maxlen=max_length , truncating=trunc_type)

# Build the model 

In [10]:
model = tf.keras.Sequential([
    
    tf.keras.layers.Embedding(vocabulary_size , embedding_dim , input_length=max_length),
    
    tf.keras.layers.Flatten(),
    
    tf.keras.layers.Dense(6 , activation = 'relu'),
    
    tf.keras.layers.Dense(1 , activation = 'sigmoid')
])

model.compile(loss = 'binary_crossentropy' , optimizer = 'adam' , metrics = ['accuracy'])

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 120, 16)           160000    
                                                                 
 flatten (Flatten)           (None, 1920)              0         
                                                                 
 dense (Dense)               (None, 6)                 11526     
                                                                 
 dense_1 (Dense)             (None, 1)                 7         
                                                                 
Total params: 171533 (670.05 KB)
Trainable params: 171533 (670.05 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [12]:
# Train the model :

num_epochs = 10

model.fit(padded , training_labels_final , epochs = num_epochs , validation_data = (test_padded , test_labels_final) )

Epoch 1/10


1531/1531 [==============================] - 17s 8ms/step - loss: 0.4297 - accuracy: 0.7885 - val_loss: 0.2686 - val_accuracy: 0.8941
Epoch 2/10
1531/1531 [==============================] - 12s 8ms/step - loss: 0.2373 - accuracy: 0.9057 - val_loss: 0.1810 - val_accuracy: 0.9382
Epoch 3/10
1531/1531 [==============================] - 11s 7ms/step - loss: 0.1004 - accuracy: 0.9690 - val_loss: 0.1606 - val_accuracy: 0.9462
Epoch 4/10
1531/1531 [==============================] - 11s 7ms/step - loss: 0.0252 - accuracy: 0.9952 - val_loss: 0.1970 - val_accuracy: 0.9471
Epoch 5/10
1531/1531 [==============================] - 7s 4ms/step - loss: 0.0054 - accuracy: 0.9994 - val_loss: 0.2299 - val_accuracy: 0.9471
Epoch 6/10
1531/1531 [==============================] - 6s 4ms/step - loss: 0.0011 - accuracy: 1.0000 - val_loss: 0.2563 - val_accuracy: 0.9482
Epoch 7/10
1531/1531 [==============================] - 7s 4ms/step - loss: 2.7351e-04 - accuracy: 1.0000 - val_loss: 0.2772 - val

# Visualize word embeddings

In [26]:
embedded_layer = model.layers[0]            # Get the embedded layer form model:

embedding_weights = embedded_layer.get_weights()[0]         #weights of embeded layer\
    
print(embedding_weights.shape)

(10000, 16)


We will need to generate two files:

vecs.tsv - contains the vector weights of each word in the vocabulary


meta.tsv - contains the words in the vocabulary

In [27]:
# Get the index word dictionary :

reverse_word_index = tokenizer.index_word

In [28]:
import io
# open writable files :

out_v = io.open('vecs.tsv' , 'w' , encoding='utf-8')

out_m = io.open('meta.tsv' , 'w' , encoding='utf-8')

# Initialize the loop. Start counting at `1` because `0` is just for the padding

for words_num in range(1 , vocabulary_size):
    
    word_name = reverse_word_index[words_num]       # word associated with index
    
    word_embedding = embedding_weights[words_num]   # Embedding weights associated with words
    
    out_m.write(word_name + '\n')                   # write the word name in file
    
    out_v.write('\t'.join([str(x) for x in word_embedding]) + '\n' )

# close the files

out_m.close()

out_v.close()
    
    

In [13]:
import pickle

pickle.dump(model , open('IMDB.pkl' , 'wb'))